In [1]:
import os
archivos_entrenamiento = os.listdir("Entrenamiento/")
archivo_test = os.listdir("Test/")
#set([os.path.splitext(archivo)[1] for archivo in archivos_entrenamiento]) # obtengo las extensiones de los archivos

In [2]:
class ArchivoTxt:
    def __init__(self, nombre, extension, texto):
        self.nombre = nombre
        self.extension = extension
        self.texto = texto

In [3]:
from tika import parser
from nltk import word_tokenize

def convertir_documento_a_txt(archivo):
    raw = parser.from_file(archivo)
    return raw['content']

#word_tokenize(convertir_pdf_txt("dataset-nlp-plagio-utn/Domótica_Final.pptx"))

In [4]:
def convertir_archivo_a_txt(archivo):
    archivo_nombre, archivo_extension = os.path.splitext(archivo)
    if(archivo_extension != ".pptx"):
        archivo_txt = convertir_documento_a_txt(archivo)
        return ArchivoTxt(archivo_nombre, archivo_extension, archivo_txt)
    
#convertir_archivo_a_txt("dataset-nlp-plagio-utn/MKT 2016 - Alan Szpigiel - TP4.pdf").nombre

In [5]:
from nltk import sent_tokenize # divide en oraciones

archivos_entrenamiento_txt = [convertir_archivo_a_txt("Entrenamiento/" + archivo) for archivo in archivos_entrenamiento]

#for archivo in archivos_entrenamiento_txt:
#    if(archivo != None):
#        archivo.texto = sent_tokenize(archivo.texto.strip())

2020-05-25 14:50:54,957 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [6]:
archivo_test_txt = [convertir_archivo_a_txt("Test/" + archivo) for archivo in archivo_test][0]

In [7]:
# Nombre del archivo de texto procesado
archivo_test_txt.nombre # la extension hace falta?

'Test/TP 3 The experience economy'

In [48]:
# Nombre y apellido del alumno
import spacy
import itertools
from nltk import line_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords 

nombre_alumno=''

nlp=spacy.load('es_core_news_sm') # modelo para detectar Entidades (nombres)
texto_dividido_en_lineas = [re.sub('[^A-Za-z0-9áéíóúñ ]+', ' ', linea) for linea in line_tokenize(archivo_test_txt.texto.strip())]
lineas_pasadas_por_el_modelo = [[(a.text, a.label_) for a in nlp(linea).ents] for linea in texto_dividido_en_lineas]
lista_flatenizada = list(itertools.chain.from_iterable(lineas_pasadas_por_el_modelo)) #flatten

entidades_reconocidas_como_personas = [texto for (texto,categoria) in lista_flatenizada if categoria == 'PER']

sw = stopwords.words('spanish')
entidades_reconocidas_como_personas = [entidad for entidad in entidades_reconocidas_como_personas if not entidad.lower().strip() in sw]

#print([pos_tag(word_tokenize(entidad)) for entidad in entidades_reconocidas_como_personas])

for entidad in entidades_reconocidas_como_personas:
    if entidad in archivo_test_txt.nombre:
        nombre_alumno = entidad

if(nombre_alumno != ''):
    print('Nombre y apellido del alumno: ' + nombre_alumno)
else: # Fijarse cual de las entidades reconocidas como personas esta en un contexto como Nombre:, Integrantes:, etc
    print(entidades_reconocidas_como_personas)
    #print([nltk.Text(word_tokenize(archivo_test_txt.texto)).concordance(entidad) for entidad in entidades_reconocidas_como_personas])

['Alejandro Prince', 'Brian Daniel Leder', 'Joseph PINE II', 'James GILMORE', 'Trabajo Práctico', 'Cuáles', '9  Qué', 'adquisición']


In [43]:
import re, math
from collections import Counter

Palabra = re.compile(r'\w+')

def obtener_similitud_del_coseno(vector1, vector2):
     interseccion = set(vector1.keys()) & set(vector2.keys())
     numerador = sum([vector1[x] * vector2[x] for x in interseccion])

     sum1 = sum([vector1[x]**2 for x in vector1.keys()])
     sum2 = sum([vector2[x]**2 for x in vector2.keys()])
     denominador = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominador:
        return 0.0
     else:
        return float(numerador) / denominador

def string_a_vector(texto):
     palabras = Palabra.findall(texto)
     return Counter(palabras)

string1 = 'Le dije: Hola como te va Raúl?'
string2 = 'Le pregunté a Raúl como estaba'

def obtener_similitud(texto1, texto2):
    vector1 = string_a_vector(texto1)
    vector2 = string_a_vector(texto2)
    return obtener_similitud_del_coseno(vector1, vector2)

print('Similitud: ', obtener_similitud(string1, string2))

Similitud:  0.4629100498862757


In [46]:
def obtener_oracion_mas_parecida(oracion):
    mayor_porcentaje = 0.0
    oracion_mas_parecida = ''
    archivo_donde_se_encontro = ''
    for archivo in archivos_entrenamiento_txt:
        if(archivo != None and obtener_similitud(archivo_test_txt.texto,archivo.texto)<0.9):
            for oracion_a_comparar in sent_tokenize(archivo.texto.strip()):
                similitud = obtener_similitud(oracion.lower(), oracion_a_comparar.lower())
                if(similitud > mayor_porcentaje):
                    mayor_porcentaje = similitud
                    oracion_mas_parecida = oracion_a_comparar
                    archivo_donde_se_encontro = archivo.nombre
                
    return (re.sub('[^A-Za-z0-9áéíóúñ:/ ]+', ' ', oracion), re.sub('[^A-Za-z0-9áéíóúñ:/ ]+', ' ', oracion_mas_parecida), mayor_porcentaje, archivo_donde_se_encontro)

In [47]:
# Obtener Plagio de otros tps
# falta ver que tp pasó primero, que porcentaje poner como limite, que las consignas no sean plagio
porcentajes_de_aparicion = [obtener_oracion_mas_parecida(oracion) for oracion in sent_tokenize(archivo_test_txt.texto.strip())]
[(oracion,posible_plagio,porcentaje,archivo) for (oracion,posible_plagio,porcentaje,archivo) in porcentajes_de_aparicion if porcentaje>0.7]

[('Marketing en Internet y Nueva Economía curso: K5052    1 Cuat ',
  'Marketing en Internet y Nueva Economía curso: K5052    1 Cuat ',
  0.9999999999999998,
  'Entrenamiento/TP4 Adopcion (1)'),
 ('2017 Profesor: Alejandro Prince Fecha de entrega: 10/04/2017 NOMBRE Y APELLIDO LEGAJO N EMAIL CONTACTO Brian Daniel Leder 146 641 0 briandleder gmail com The experience economy   Joseph PINE II y James GILMORE Preguntas de Trabajo Práctico NOTA: no es requerida la lectura ni recordación desde la página 17 inclusive hasta la 20 del texto ',
  'Alumno: Andrés Basso                       TP 3: Economía de experiencia Legajo: 124 184 9 The experience economy   Joseph PINE II y James GILMORE Preguntas de Trabajo Práctico NOTA: no es requerida la lectura ni recordación desde la página 17 inclusive hasta la 20 del texto ',
  0.708010432393843,
  'Entrenamiento/TP 3 - Economía de Experiencia - Andrés Basso (1)'),
 ('Es obligatoria desde la 1ra hasta la página 16 ',
  'Es obligatoria desde la 1ra has